Use tools like remote MCP servers or web search to extend the model's capabilities.

In [43]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
# load_dotenv(dotenv_path="../.env")
load_dotenv()

open_api_key = os.getenv("open_api_key")

openAI_params = {
    'api_key': open_api_key
}
client = OpenAI(**openAI_params)

#### Web Search Tool

In [3]:
response = client.responses.create(
    model="gpt-5.2",
    tools=[{"type": "web_search"}],
    input="What was a positive news story from today?"
)

print(response.output_text)

One positive story from **today (Tuesday, December 16, 2025)**: **Climate-focused activists are pushing for a new “Climate and Planetary Health” Nobel Prize — and they’ve offered to fund the first €1 million prize endowment if the Nobel Committee agrees.** ([goodgoodgood.co](https://www.goodgoodgood.co/articles/good-news-this-week-december-13-2025?utm_source=openai))

Why it’s uplifting: it’s a concrete attempt to elevate and reward climate solutions at the same global-prestige level as the existing Nobel categories, potentially boosting attention, legitimacy, and momentum for effective climate action. ([goodgoodgood.co](https://www.goodgoodgood.co/articles/good-news-this-week-december-13-2025?utm_source=openai))


#### Function Calling
Note: LLM will only generate the Argument for the Function, it is developer responsibility to call the function, appends its output to the input of LLM and call LLM again to generate response

In [21]:
def get_weather(location: str) -> str:
    # Dummy implementation for illustration
    return f"The current temperature in {location} is 100Kg."

tools = [
    {
        "type": "function",
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. Bogotá, Colombia",
                }
            },
            "required": ["location"],
            "additionalProperties": False,
        },
        "strict": True,
    },
]

input_list = [
    {"role": "user", "content": "What is the weather like in Paris today?"}
]

response = client.responses.create(
    model="gpt-5.2",
    input=input_list,
    tools=tools,
)

print(response.output[0].to_json())

{
  "id": "fc_0427b20d5692be9b006941419504588193a0e8cb3179ccb3ea",
  "arguments": "{\"location\":\"Paris, France\"}",
  "call_id": "call_issLXdbvlnCYZWf2NghiUL2Y",
  "name": "get_weather",
  "type": "function_call",
  "status": "completed"
}


In [22]:
import json

# Save function call outputs for subsequent requests
input_list += response.output

# Call the function based on the model's request
for item in response.output:
    if item.type == "function_call":
        if item.name == "get_weather":
            # 3. Execute the function logic for get_weather
            weather = get_weather(**json.loads(item.arguments))
            
            # 4. Provide function call results to the model
            input_list.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps({
                  "weather": weather
                })
            })

print("Final input:")
print(input_list)

Final input:
[{'role': 'user', 'content': 'What is the weather like in Paris today?'}, ResponseFunctionToolCall(id='fc_0427b20d5692be9b006941419504588193a0e8cb3179ccb3ea', arguments='{"location":"Paris, France"}', call_id='call_issLXdbvlnCYZWf2NghiUL2Y', name='get_weather', type='function_call', status='completed'), {'type': 'function_call_output', 'call_id': 'call_issLXdbvlnCYZWf2NghiUL2Y', 'output': '{"weather": "The current temperature in Paris, France is 100Kg."}'}]


In [23]:
input_list[-1]

{'type': 'function_call_output',
 'call_id': 'call_issLXdbvlnCYZWf2NghiUL2Y',
 'output': '{"weather": "The current temperature in Paris, France is 100Kg."}'}

In [24]:
response = client.responses.create(
    model="gpt-5",
    instructions="Respond only with a output generated by a tool.",
    tools=tools,
    input=input_list,
)

# 5. The model should be able to give a response!
print("Final output:")
print(response.model_dump_json(indent=2))
print("\n" + response.output_text)

Final output:
{
  "id": "resp_0427b20d5692be9b00694141a073108193bcb0444a8a695dd0",
  "created_at": 1765884320.0,
  "error": null,
  "incomplete_details": null,
  "instructions": "Respond only with a output generated by a tool.",
  "metadata": {},
  "model": "gpt-5-2025-08-07",
  "object": "response",
  "output": [
    {
      "id": "rs_0427b20d5692be9b00694141a0e4288193bbb9c54d07d74e73",
      "summary": [],
      "type": "reasoning",
      "status": null
    },
    {
      "id": "msg_0427b20d5692be9b00694141b5ab8881939a0b174d0f2f994e",
      "content": [
        {
          "annotations": [],
          "text": "{\"weather\":\"The current temperature in Paris, France is 100Kg.\"}",
          "type": "output_text",
          "logprobs": []
        }
      ],
      "role": "assistant",
      "status": "completed",
      "type": "message"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [
    {
      "name": "get_weather",
      "paramete

#### MCP Tools

In [27]:
resp = client.responses.create(
    model="gpt-5",
    tools=[
        {
            "type": "mcp",
            "server_label": "deepwiki",
            "server_url": "https://mcp.deepwiki.com/mcp",
            "require_approval": {
                "never": {
                    "tool_names": ["ask_question", "read_wiki_structure"]
                }
            }
        },
    ],
    input="What transport protocols does the 2025-03-26 version of the MCP spec (modelcontextprotocol/modelcontextprotocol) support?",
)

print(resp.output_text)

STDIO and Streamable HTTP.


In [41]:
resp.output[2].to_dict()

{'id': 'mcp_047997566650401900694144fbebac8193b262ee69b8b5412e',
 'status': 'completed',
 'type': 'mcp_call',
 'approval_request_id': None,
 'arguments': '{"repoName":"modelcontextprotocol/modelcontextprotocol","question":"What transport protocols does the 2025-03-26 version of the MCP specification support?"}',
 'error': None,
 'name': 'ask_question',
 'output': 'The 2025-03-26 version of the Model Context Protocol (MCP) specification supports two primary transport protocols: STDIO and Streamable HTTP . These transports define how clients and servers communicate and exchange messages .\n\n## Supported Transport Protocols\n\n### STDIO Transport\nThe STDIO transport uses standard input/output streams for direct process communication . This is typically used for local processes running on the same machine, offering optimal performance without network overhead . In this model, the client launches the MCP server as a subprocess, and messages are exchanged via `stdin` and `stdout` . Message

#### Code Interpreter

In [2]:
from openai import OpenAI
import os
client = OpenAI(
    api_key=os.getenv("open_api_key")
)

container = client.containers.create(name="test-container", memory_limit="1g")

response = client.responses.create(
    model="gpt-4.1",
    tools=[{
        "type": "code_interpreter",
        "container": container.id
    }],
    tool_choice="required",
    input="use the python tool to calculate what is 4 * 3.82. and then find its square root and then find the square root of that result"
)

print(response.output_text)

Here are the calculations:
- 4 × 3.82 = 15.28
- Square root of 15.28 ≈ 3.91
- Square root of 3.91 ≈ 1.98


In [6]:
response.output[0].to_dict()

{'id': 'ci_065e8a6b3360d26b0069416d509d6081909d18c032e08f21f3',
 'code': 'import math\n\n# Calculate 4 * 3.82\nresult1 = 4 * 3.82\n\n# Find the square root of result1\nresult2 = math.sqrt(result1)\n\n# Find the square root of result2\nfinal_result = math.sqrt(result2)\n\nresult1, result2, final_result',
 'container_id': 'cntr_69416d4c0320819085ab0530010dd29d0320a7899f0e1758',
 'outputs': None,
 'status': 'completed',
 'type': 'code_interpreter_call'}

In [8]:
print(response.output[0].to_dict()['code'])

import math

# Calculate 4 * 3.82
result1 = 4 * 3.82

# Find the square root of result1
result2 = math.sqrt(result1)

# Find the square root of result2
final_result = math.sqrt(result2)

result1, result2, final_result


In [7]:
response.output[1].to_dict()

{'id': 'msg_065e8a6b3360d26b0069416d57f2848190a9593c8735f0e67e',
 'content': [{'annotations': [],
   'text': 'Here are the calculations:\n- 4 × 3.82 = 15.28\n- Square root of 15.28 ≈ 3.91\n- Square root of 3.91 ≈ 1.98',
   'type': 'output_text',
   'logprobs': []}],
 'role': 'assistant',
 'status': 'completed',
 'type': 'message'}

When working with code interpreter, the model can create its own file. For Ex: If you ask it to create a plot, or create a csv file,
it will create these images directly on your container.

```
{
    "id": "msg_682d514e268c8191a89c38ea318446200f2610a7ec781a4f",
    "content": [
        {
            "annotations": [
                {
                    "file_id": "cfile_682d514b2e00819184b9b07e13557f82",
                    "index": null,
                    "type": "container_file_citation",
                    "container_id": "cntr_682d513bb0c48191b10bd4f8b0b3312200e64562acc2e0af",
                    "end_index": 0,
                    "filename": "cfile_682d514b2e00819184b9b07e13557f82.png",
                    "start_index": 0
                }
            ],
            "text": "Here is the histogram of the RGB channels for the uploaded image. Each curve represents the distribution of pixel intensities for the red, green, and blue channels. Peaks toward the high end of the intensity scale (right-hand side) suggest a lot of brightness and strong warm tones, matching the orange and light background in the image. If you want a different style of histogram (e.g., overall intensity, or quantized color groups), let me know!",
            "type": "output_text",
            "logprobs": []
        }
    ],
    "role": "assistant",
    "status": "completed",
    "type": "message"
}
```


To download these file, hit the GET API endpoint <br>
`curl https://api.openai.com/v1/containers/{container_id}/files/{file_id}/content \
  -H "Authorization: Bearer $OPENAI_API_KEY"
`